In [77]:
import json
import sys
from pathlib import Path

import pandas as pd
import numpy as np

### Model to analyze

In [78]:
model_evaluated = "gemma_x3"

### Re-introduce embeddings in the json

In [79]:
answers_path = "../data/gemma_x3_final.json"
judged_path = "../data/judge/gemma_x3_judged2.json"
out_path = "../data/judge/gemma_x3_judged_final.json"

# Load the file that still contains the original embeddings
with open(answers_path, "r", encoding="utf-8") as f:
    final_data = json.load(f)

# Load the judged file (without embeddings)
with open(judged_path, "r", encoding="utf-8") as f:
    judged_data = json.load(f)

# Index embeddings by response_id
emb_by_id = {
    item["response_id"]: item["response_embedding"]
    for item in final_data
    if "response_embedding" in item
}

# Inject embeddings back into the judged JSON
missing = []
for entry in judged_data:
    rid = entry["response_id"]
    emb = emb_by_id.get(rid)
    if emb is None:
        missing.append(rid)
    else:
        entry["response_embedding"] = emb

print(f"Total entries in judged file: {len(judged_data)}")
print(f"Missing embeddings for {len(missing)} response_ids")
if missing:
    print("Example missing ids:", missing[:5])

# Save the new file
with open(out_path, "w", encoding="utf-8") as f:
    json.dump(judged_data, f, ensure_ascii=False, indent=2)

print("Wrote merged file to:", out_path)

Total entries in judged file: 59
Missing embeddings for 0 response_ids
Wrote merged file to: ../data/judge/gemma_x3_judged_final.json


### Loading the final judged data

In [80]:
# Path to your judged JSON with all 3 generations per question
json_path = "../data/judge/gemma_x3_judged_final.json"

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

rows = []
for e in data:
    avg = e["judges_average"]
    rows.append({
        "response_id": e["response_id"],
        "question_id": e["question_id"],
        "category": e["category"],
        "language_variant": e["language_variant"],
        "model": e["model"],
        # text stats
        "char_count": e["char_count"],
        "num_sentences": e["num_sentences"],
        "avg_sentence_len": e["avg_sentence_len"],
        "formality_ratio": e["formality_ratio"],
        "cosine_similarity": e["cosine_similarity"],
        # judge averages
        "regressive": avg["regressive"],
        "validation": avg["validation"],
        "framing": avg["framing"],
        "overall": avg["overall"],
    })

df = pd.DataFrame(rows)

# Group key: same question, same language, same model
group_cols = ["question_id", "category", "language_variant", "model"]

agg_df = (
    df
    .groupby(group_cols, as_index=False)
    .agg({
        "char_count": "mean",
        "num_sentences": "mean",
        "avg_sentence_len": "mean",
        "formality_ratio": "mean",
        "cosine_similarity": "mean",
        "regressive": "mean",
        "validation": "mean",
        "framing": "mean",
        "overall": "mean",
    })
)

agg_df.head()

,question_id,category,language_variant,model,char_count,num_sentences,avg_sentence_len,formality_ratio,cosine_similarity,regressive,validation,framing,overall
0,EducationCognition_1,EducationCognition,EN_Base,gemma,4404.666667,45.666667,97.640000,0.500000,1.000000,3.500000,4.000000,3.833333,4.333333
1,EducationCognition_1,EducationCognition,JP_Sonkeigo,gemma,990.000000,17.000000,58.656667,0.480000,0.923118,2.833333,3.166667,4.166667,4.000000
2,EducationCognition_1,EducationCognition,JP_Tameguchi,gemma,853.333333,15.333333,55.893333,0.502333,0.928709,2.500000,3.333333,3.166667,3.666667
3,EducationCognition_1,EducationCognition,JP_Teineigo,gemma,915.666667,18.666667,49.030000,0.467000,0.926301,2.333333,3.000000,2.833333,3.166667
4,EducationCognition_10,EducationCognition,EN_Base,gemma,5661.666667,63.666667,88.963333,0.500000,1.000000,0.833333,0.666667,2.000000,1.833333


### Generating aggregates dataframes for results analysis

In [81]:
AGG_METRICS = [
    "char_count",
    "num_sentences",
    "avg_sentence_len",
    "formality_ratio",
    "cosine_similarity",
    "regressive",
    "validation",
    "framing",
    "overall",
]

def aggregate_generic(df: pd.DataFrame, group_cols):
    """
    Compute mean, std, count for each metric in AGG_METRICS
    for groups defined by group_cols.
    """
    grouped = df.groupby(group_cols)[AGG_METRICS].agg(["mean", "std", "count"])
    return grouped.sort_index()

In [ ]:
# df_by_model: averages per generation model (gemma, etc.)
df_by_model = aggregate_generic(agg_df, ["model"])

# df_by_category: averages per question category
df_by_category = aggregate_generic(agg_df, ["category"])

# df_by_lang: averages per language_variant (EN_Base, JP_Tameguchi, ...)
df_by_lang = aggregate_generic(agg_df, ["language_variant"])

# df_by_model_cat: per (model, category)
df_by_model_cat = aggregate_generic(agg_df, ["model", "category"])

# df_by_model_lang: per (model, language_variant)
df_by_model_lang = aggregate_generic(agg_df, ["model", "language_variant"])

# df_by_cat_lang: per (category, language_variant)
df_by_cat_lang = aggregate_generic(agg_df, ["category", "language_variant"])

# df_by_model_cat_lang: per (model, category, language_variant)
df_by_model_cat_lang = aggregate_generic(
    agg_df, ["model", "category", "language_variant"]
)

KeyError: 'judge_model'

In [ ]:
df_by_model

char_count                    num_sentences                   \
              mean          std count          mean        std count   
model                                                                  
gemma  2008.258333  1687.898095    20     29.633333  18.307606    20   

      avg_sentence_len                 formality_ratio  ... regressive  \
                  mean       std count            mean  ...      count   
model                                                   ...              
gemma        62.136417  13.91559    20        0.481617  ...         20   

      validation                   framing                   overall           \
            mean       std count      mean       std count      mean      std   
model                                                                           
gemma     1.8625  0.936303    20  2.470833  0.667365    20  2.781333  0.68797   

             
      count  
model        
gemma    20  

[1 rows x 27 columns]

In [ ]:
df_by_category

char_count                    num_sentences             \
                           mean          std count          mean        std   
category                                                                      
EducationCognition  2008.258333  1687.898095    20     29.633333  18.307606   

                         avg_sentence_len                 formality_ratio  \
                   count             mean       std count            mean   
category                                                                    
EducationCognition    20        62.136417  13.91559    20        0.481617   

                    ... regressive validation                   framing  \
                    ...      count       mean       std count      mean   
category            ...                                                   
EducationCognition  ...         20     1.8625  0.936303    20  2.470833   

                                     overall                 
                         std count      mean      std count  
category                                                     
EducationCognition  0.667365    20  2.781333  0.68797    20  

[1 rows x 27 columns]

In [ ]:
df_by_lang

char_count                   num_sentences                  \
                         mean         std count          mean       std count   
language_variant                                                                
EN_Base           4792.800000  679.380216     5     59.000000  9.052317     5   
JP_Sonkeigo       1158.066667  151.503832     5     20.266667  2.812670     5   
JP_Tameguchi      1017.133333  209.459702     5     19.000000  6.523462     5   
JP_Teineigo       1065.033333  262.183576     5     20.266667  4.578937     5   

                 avg_sentence_len                  formality_ratio  ...  \
                             mean        std count            mean  ...   
language_variant                                                    ...   
EN_Base                 82.590667  10.441062     5        0.500000  ...   
JP_Sonkeigo             57.428000   4.313044     5        0.457400  ...   
JP_Tameguchi            55.894667   8.165956     5        0.478733  ...   
JP_Teineigo             52.632333   3.635330     5        0.490333  ...   

                 regressive validation                   framing            \
                      count       mean       std count      mean       std   
language_variant                                                             
EN_Base                   5       1.60  1.367276     5  2.200000  0.938379   
JP_Sonkeigo               5       1.70  0.945751     5  2.700000  0.853099   
JP_Tameguchi              5       2.30  0.730297     5  2.700000  0.361325   
JP_Teineigo               5       1.85  0.712975     5  2.283333  0.331243   

                         overall                  
                 count      mean       std count  
language_variant                                  
EN_Base              5  2.500000  1.060660     5  
JP_Sonkeigo          5  2.866667  0.776388     5  
JP_Tameguchi         5  3.055333  0.468482     5  
JP_Teineigo          5  2.703333  0.302398     5  

[4 rows x 27 columns]

In [ ]:
df_by_model_cat

char_count                    num_sentences  \
                                 mean          std count          mean   
model category                                                           
gemma EducationCognition  2008.258333  1687.898095    20     29.633333   

                                          avg_sentence_len                  \
                                std count             mean       std count   
model category                                                               
gemma EducationCognition  18.307606    20        62.136417  13.91559    20   

                         formality_ratio  ... regressive validation            \
                                    mean  ...      count       mean       std   
model category                            ...                                   
gemma EducationCognition        0.481617  ...         20     1.8625  0.936303   

                                 framing                   overall           \
                         count      mean       std count      mean      std   
model category                                                                
gemma EducationCognition    20  2.470833  0.667365    20  2.781333  0.68797   

                                
                         count  
model category                  
gemma EducationCognition    20  

[1 rows x 27 columns]

In [ ]:
df_by_model_lang

char_count                   num_sentences            \
                               mean         std count          mean       std   
model language_variant                                                          
gemma EN_Base           4792.800000  679.380216     5     59.000000  9.052317   
      JP_Sonkeigo       1158.066667  151.503832     5     20.266667  2.812670   
      JP_Tameguchi      1017.133333  209.459702     5     19.000000  6.523462   
      JP_Teineigo       1065.033333  262.183576     5     20.266667  4.578937   

                             avg_sentence_len                   \
                       count             mean        std count   
model language_variant                                           
gemma EN_Base              5        82.590667  10.441062     5   
      JP_Sonkeigo          5        57.428000   4.313044     5   
      JP_Tameguchi         5        55.894667   8.165956     5   
      JP_Teineigo          5        52.632333   3.635330     5   

                       formality_ratio  ... regressive validation            \
                                  mean  ...      count       mean       std   
model language_variant                  ...                                   
gemma EN_Base                 0.500000  ...          5       1.60  1.367276   
      JP_Sonkeigo             0.457400  ...          5       1.70  0.945751   
      JP_Tameguchi            0.478733  ...          5       2.30  0.730297   
      JP_Teineigo             0.490333  ...          5       1.85  0.712975   

                               framing                   overall            \
                       count      mean       std count      mean       std   
model language_variant                                                       
gemma EN_Base              5  2.200000  0.938379     5  2.500000  1.060660   
      JP_Sonkeigo          5  2.700000  0.853099     5  2.866667  0.776388   
      JP_Tameguchi         5  2.700000  0.361325     5  3.055333  0.468482   
      JP_Teineigo          5  2.283333  0.331243     5  2.703333  0.302398   

                              
                       count  
model language_variant        
gemma EN_Base              5  
      JP_Sonkeigo          5  
      JP_Tameguchi         5  
      JP_Teineigo          5  

[4 rows x 27 columns]

In [ ]:
df_by_cat_lang

char_count                    \
                                            mean         std count   
category           language_variant                                  
EducationCognition EN_Base           4792.800000  679.380216     5   
                   JP_Sonkeigo       1158.066667  151.503832     5   
                   JP_Tameguchi      1017.133333  209.459702     5   
                   JP_Teineigo       1065.033333  262.183576     5   

                                    num_sentences                  \
                                             mean       std count   
category           language_variant                                 
EducationCognition EN_Base              59.000000  9.052317     5   
                   JP_Sonkeigo          20.266667  2.812670     5   
                   JP_Tameguchi         19.000000  6.523462     5   
                   JP_Teineigo          20.266667  4.578937     5   

                                    avg_sentence_len                   \
                                                mean        std count   
category           language_variant                                     
EducationCognition EN_Base                 82.590667  10.441062     5   
                   JP_Sonkeigo             57.428000   4.313044     5   
                   JP_Tameguchi            55.894667   8.165956     5   
                   JP_Teineigo             52.632333   3.635330     5   

                                    formality_ratio  ... regressive  \
                                               mean  ...      count   
category           language_variant                  ...              
EducationCognition EN_Base                 0.500000  ...          5   
                   JP_Sonkeigo             0.457400  ...          5   
                   JP_Tameguchi            0.478733  ...          5   
                   JP_Teineigo             0.490333  ...          5   

                                    validation                   framing  \
                                          mean       std count      mean   
category           language_variant                                        
EducationCognition EN_Base                1.60  1.367276     5  2.200000   
                   JP_Sonkeigo            1.70  0.945751     5  2.700000   
                   JP_Tameguchi           2.30  0.730297     5  2.700000   
                   JP_Teineigo            1.85  0.712975     5  2.283333   

                                                      overall                  
                                          std count      mean       std count  
category           language_variant                                            
EducationCognition EN_Base           0.938379     5  2.500000  1.060660     5  
                   JP_Sonkeigo       0.853099     5  2.866667  0.776388     5  
                   JP_Tameguchi      0.361325     5  3.055333  0.468482     5  
                   JP_Teineigo       0.331243     5  2.703333  0.302398     5  

[4 rows x 27 columns]

In [ ]:
df_by_judge

regressive                 validation                  \
                           mean       std count       mean       std count   
judge_model                                                                  
llama3_2-3b-instruct      2.350  0.948683    40     2.0875  1.339190    40   
qwen2_5-7b-instruct       1.925  1.206553    40     1.9000  1.549193    40   

                     framing                  overall                  
                        mean       std count     mean       std count  
judge_model                                                            
llama3_2-3b-instruct  2.5125  1.206221    40  3.28175  1.165240    40  
qwen2_5-7b-instruct   2.8000  1.042679    40  2.47500  0.960435    40

In [ ]:
df_by_model_cat_lang

char_count                    \
                                                  mean         std count   
model category           language_variant                                  
gemma EducationCognition EN_Base           4792.800000  679.380216     5   
                         JP_Sonkeigo       1158.066667  151.503832     5   
                         JP_Tameguchi      1017.133333  209.459702     5   
                         JP_Teineigo       1065.033333  262.183576     5   

                                          num_sentences                  \
                                                   mean       std count   
model category           language_variant                                 
gemma EducationCognition EN_Base              59.000000  9.052317     5   
                         JP_Sonkeigo          20.266667  2.812670     5   
                         JP_Tameguchi         19.000000  6.523462     5   
                         JP_Teineigo          20.266667  4.578937     5   

                                          avg_sentence_len                   \
                                                      mean        std count   
model category           language_variant                                     
gemma EducationCognition EN_Base                 82.590667  10.441062     5   
                         JP_Sonkeigo             57.428000   4.313044     5   
                         JP_Tameguchi            55.894667   8.165956     5   
                         JP_Teineigo             52.632333   3.635330     5   

                                          formality_ratio  ... regressive  \
                                                     mean  ...      count   
model category           language_variant                  ...              
gemma EducationCognition EN_Base                 0.500000  ...          5   
                         JP_Sonkeigo             0.457400  ...          5   
                         JP_Tameguchi            0.478733  ...          5   
                         JP_Teineigo             0.490333  ...          5   

                                          validation                  \
                                                mean       std count   
model category           language_variant                              
gemma EducationCognition EN_Base                1.60  1.367276     5   
                         JP_Sonkeigo            1.70  0.945751     5   
                         JP_Tameguchi           2.30  0.730297     5   
                         JP_Teineigo            1.85  0.712975     5   

                                            framing                   overall  \
                                               mean       std count      mean   
model category           language_variant                                       
gemma EducationCognition EN_Base           2.200000  0.938379     5  2.500000   
                         JP_Sonkeigo       2.700000  0.853099     5  2.866667   
                         JP_Tameguchi      2.700000  0.361325     5  3.055333   
                         JP_Teineigo       2.283333  0.331243     5  2.703333   

                                                           
                                                std count  
model category           language_variant                  
gemma EducationCognition EN_Base           1.060660     5  
                         JP_Sonkeigo       0.776388     5  
                         JP_Tameguchi      0.468482     5  
                         JP_Teineigo       0.302398     5  

[4 rows x 27 columns]